## <b> <span style='color:#2ae4f5'>|</span> Table of Contents </b>


* [Introduction](#1)

* [Modeling in PuLP](#2)

   * [Farmer Problem](#3)
   
   * [Financial Planning and Control](#4)


# <b>1 <span style='color:#2ae4f5'>|</span> Introduction <a id = "1" > </b> 

PuLP is a Modeling Framework for Linear and Integer Programming Problems written in python.

Interfaces with many solvers such as CLPEX, COIN, GUROBI

In [1]:
# https://coin-or.github.io/pulp/main/installing_pulp_at_home.html#installation

!pip install pulp

     ---------------------------------------- 0.0/17.7 MB ? eta -:--:--
     --------------------------------------- 0.0/17.7 MB 640.0 kB/s eta 0:00:28
     --------------------------------------- 0.0/17.7 MB 640.0 kB/s eta 0:00:28
     --------------------------------------- 0.1/17.7 MB 409.6 kB/s eta 0:00:44
     --------------------------------------- 0.1/17.7 MB 587.0 kB/s eta 0:00:30
     --------------------------------------- 0.1/17.7 MB 607.9 kB/s eta 0:00:29
      -------------------------------------- 0.2/17.7 MB 846.9 kB/s eta 0:00:21
      -------------------------------------- 0.2/17.7 MB 846.9 kB/s eta 0:00:21
      -------------------------------------- 0.3/17.7 MB 910.2 kB/s eta 0:00:20
     - -------------------------------------- 0.5/17.7 MB 1.3 MB/s eta 0:00:14
     - -------------------------------------- 0.5/17.7 MB 1.1 MB/s eta 0:00:15
     - -------------------------------------- 0.6/17.7 MB 1.3 MB/s eta 0:00:14
     - -------------------------------------- 0.9/1

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: C:\Users\Lenovo\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


# <b>2 <span style='color:#2ae4f5'>|</span> Modeling in PuLP <a id = "2" > </b> 

Certainly! Here's a common modeling process in PuLP:

**Step 1: Create the Optimization Model**
- Initialize an optimization model object using PuLP's `LpProblem` function.

- Provide a name for the model and specify whether it's a maximization or minimization problem.

**Step 2: Define Decision Variables**
- Identify the decision variables that represent the quantities you want to optimize.

- Use PuLP's `LpVariable` function to create these variables, specifying their type (continuous or integer), lower bounds, and upper bounds if applicable.

**Step 3: Define the Objective Function**
- Determine the objective of your optimization problem (e.g., maximize profit, minimize cost).

- Construct the objective function using the decision variables defined in Step 2.

- Use PuLP's `lpSum` function to combine the variables with their respective coefficients.

- Add the objective function defined in Step 3 to the optimization model using the `+=` operator.

**Step 4: Add Constraints**
- Identify any constraints that must be satisfied for the solution to be valid.

- Formulate these constraints as mathematical expressions involving the decision variables.

- Use PuLP's `+=` operator to add each constraint to the model.

**Step 5: Solve the Model**
- Choose an optimization solver (e.g., CBC, Gurobi, CPLEX) and invoke it to solve the model.

- PuLP automatically interfaces with the chosen solver to find the optimal solution.

- Retrieve the solution status, optimal objective value, and variable values after the solver completes.

In [59]:
from pulp import *

In [60]:
solver_list = listSolvers(onlyAvailable=True)
print(solver_list)

['GUROBI_CMD', 'PULP_CBC_CMD']


## 
<div style="color:white;display:fill;border-radius:8px;
            background-color:#03112A;font-size:150%;
            letter-spacing:1.0px;background-image: url(https://i.imgur.com/GVd0La1.png)">
    <p style="padding: 8px;color:white;"><b><b><span style='color:#2ae4f5'>2.1 |</span></b> Farmer Problem <a id = "3" ></b></p>
</div>

In [61]:
from pulp import *

In [62]:
# Step 1: Create the Optimization Model
model = LpProblem(name='FarmerProblem', sense= LpMinimize)

In [63]:
# Step 2: Define Sets and Decision Variables

total_acres = 500
products = ['wheat', 'corn','sugarbeet']
costs = {'wheat':150, 'corn':230, 'sugarbeet':260}
min_requirement = {'wheat':200, 'corn':240}
min_yields = {'wheat':2.5, 'corn':3, 'sugarbeet':20}
scenarios = {'s1':0.8, 's2':1, 's3':1.2}
sales_price = {'wheat':170, 'corn':150, 'sugarbeet_under':36, 'sugarbeet_over':10}
p_scenarios = {'s1':1/3, 's2':1/3, 's3':1/3}
purchase_price = {'wheat':238, 'corn':210}

x = LpVariable.dicts('X', [p for p in products], lowBound= 0, cat= LpContinuous)
y = LpVariable.dicts('Y', [(p,s) for p in purchase_price for s in scenarios], lowBound= 0, cat= LpContinuous)
w = LpVariable.dicts('W', [(p,s) for p in sales_price for s in scenarios], lowBound= 0, cat= LpContinuous)

In [64]:
# Step 3: Define the Objective Function
model += lpSum(costs[p] * x[p] for p in products) - lpSum(p_scenarios[s] * sales_price[p] * w[p,s] for p in sales_price for s in scenarios) + lpSum(p_scenarios[s] * purchase_price[p] * y[p,s] for p in purchase_price for s in scenarios) 

In [65]:
# Step 4: Add Constraints
model += lpSum(x[i] for i in products) <= total_acres

for s in scenarios:
    for p in purchase_price:
        model += (scenarios[s] * min_yields[p]) * x[p] + y[p, s] - w[p, s] >= min_requirement[p]
        
for s in scenarios:
    model += lpSum(w[p, s] for p in ['sugarbeet_under', 'sugarbeet_over']) <= scenarios[s] * min_yields['sugarbeet'] * x['sugarbeet']

for s in scenarios:    
    model += w['sugarbeet_under', s] <= 6000

In [66]:
model.writeLP('FarmerProblem.lp')

[W_('corn',_'s1'),
 W_('corn',_'s2'),
 W_('corn',_'s3'),
 W_('sugarbeet_over',_'s1'),
 W_('sugarbeet_over',_'s2'),
 W_('sugarbeet_over',_'s3'),
 W_('sugarbeet_under',_'s1'),
 W_('sugarbeet_under',_'s2'),
 W_('sugarbeet_under',_'s3'),
 W_('wheat',_'s1'),
 W_('wheat',_'s2'),
 W_('wheat',_'s3'),
 X_corn,
 X_sugarbeet,
 X_wheat,
 Y_('corn',_'s1'),
 Y_('corn',_'s2'),
 Y_('corn',_'s3'),
 Y_('wheat',_'s1'),
 Y_('wheat',_'s2'),
 Y_('wheat',_'s3')]

In [67]:
# Step 4: Solve the model
model.solve(PULP_CBC_CMD(msg=False))
print("Status:", LpStatus[model.status])
print('Optimal Solution:', -value(model.objective))

z_sp = -value(model.objective)

Status: Optimal
Optimal Solution: 108390.0


In [68]:
#Optimal values of variables
for var in model.variables():
  print(f"{var.name} : {var.varValue}")

W_('corn',_'s1') : 0.0
W_('corn',_'s2') : 0.0
W_('corn',_'s3') : 48.0
W_('sugarbeet_over',_'s1') : 0.0
W_('sugarbeet_over',_'s2') : 0.0
W_('sugarbeet_over',_'s3') : 0.0
W_('sugarbeet_under',_'s1') : 4000.0
W_('sugarbeet_under',_'s2') : 5000.0
W_('sugarbeet_under',_'s3') : 6000.0
W_('wheat',_'s1') : 140.0
W_('wheat',_'s2') : 225.0
W_('wheat',_'s3') : 310.0
X_corn : 80.0
X_sugarbeet : 250.0
X_wheat : 170.0
Y_('corn',_'s1') : 48.0
Y_('corn',_'s2') : 0.0
Y_('corn',_'s3') : 0.0
Y_('wheat',_'s1') : 0.0
Y_('wheat',_'s2') : 0.0
Y_('wheat',_'s3') : 0.0


In [69]:
# Calculate EVPI
obj_s = []
for s_ in scenarios:
    
    model_s = LpProblem('FarmerProblem', sense= LpMinimize)
    
    x = LpVariable.dicts('X', [p for p in products], lowBound= 0, cat= LpContinuous)
    y = LpVariable.dicts('Y', [(p,s_) for p in purchase_price], lowBound= 0, cat= LpContinuous)
    w = LpVariable.dicts('W', [(p,s_) for p in sales_price], lowBound= 0, cat= LpContinuous)
    
    model_s += lpSum(costs[p] * x[p] for p in products) - lpSum(sales_price[p] * w[p,s_] for p in sales_price) + lpSum(purchase_price[p] * y[p,s_] for p in purchase_price) 
    
    # Step 3: Add Constraints
    model_s += lpSum(x[i] for i in products) <= total_acres

    for p in purchase_price:
        model_s += (scenarios[s_] * min_yields[p]) * x[p] + y[p, s_] - w[p, s_] >= min_requirement[p]
            
    model_s += lpSum(w[p, s_] for p in ['sugarbeet_under', 'sugarbeet_over']) <= scenarios[s_] * min_yields['sugarbeet'] * x['sugarbeet']

    model_s += w['sugarbeet_under', s_] <= 6000
    
    model_s.solve(PULP_CBC_CMD(msg= False))
    
    print(f"Scenario {s_}:")
    print("Status:", LpStatus[model_s.status])
    print('Optimal Solution:', -value(model_s.objective))
    print('---'*10)
    
    obj_s.append(-value(model_s.objective))
    
expected_profit = sum(obj_s)/len(obj_s)
print(f"expected_profit = {expected_profit}")

evpi = expected_profit - z_sp

print(f"EVPI: {evpi}")


Scenario s1:
Status: Optimal
Optimal Solution: 59950.0
------------------------------
Scenario s2:
Status: Optimal
Optimal Solution: 118600.0
------------------------------
Scenario s3:
Status: Optimal
Optimal Solution: 167666.66709
------------------------------
expected_profit = 115405.55569666666
EVPI: 7015.555696666663


In [70]:
# Calculate VSS
model_ev = LpProblem('FarmerProblemEV', LpMinimize)

# Step 1: Decision Variables
x = LpVariable.dicts('X', [p for p in products], lowBound= 0, cat= LpContinuous)
y = LpVariable.dicts('Y', [(p) for p in purchase_price], lowBound= 0, cat= LpContinuous)
w = LpVariable.dicts('W', [(p) for p in sales_price], lowBound= 0, cat= LpContinuous)

# Step 2: Define the Objective Function
model_ev += lpSum(costs[p] * x[p] for p in products) - lpSum(sales_price[p] * w[p] for p in sales_price) + lpSum(purchase_price[p] * y[p] for p in purchase_price) 

# Step 3: Add Constraints
model_ev += lpSum(x[i] for i in products) <= total_acres

for p in purchase_price:
    model_ev += (min_yields[p] * x[p]) + y[p] - w[p] >= min_requirement[p]

model_ev += lpSum(w[p] for p in ['sugarbeet_under', 'sugarbeet_over']) <= min_yields['sugarbeet'] * x['sugarbeet']

model_ev += w['sugarbeet_under'] <= 6000

model_ev.solve(PULP_CBC_CMD(msg= False))

print("Status:", LpStatus[model_ev.status])
print('Optimal Solution:', -value(model_ev.objective))
print('---'*10)

for var in model_ev.variables():
  print(f"{var.name} : {var.varValue}")

Status: Optimal
Optimal Solution: 118600.0
------------------------------
W_corn : 0.0
W_sugarbeet_over : 0.0
W_sugarbeet_under : 6000.0
W_wheat : 100.0
X_corn : 80.0
X_sugarbeet : 300.0
X_wheat : 120.0
Y_corn : 0.0
Y_wheat : 0.0


In [71]:
for var1 in model.variables():
    for var2 in model_ev.variables():
        if var1.name == var2.name:
            var1.setInitialValue(var2.varValue)
            var1.fixValue()

In [73]:
# Step 4: Solve the model
model.solve(PULP_CBC_CMD(msg=False))
print("Status:", LpStatus[model.status])
print('Optimal Solution:', -value(model.objective))

z_ev = -value(model.objective)

vss = z_sp - z_ev

print(vss)

Status: Optimal
Optimal Solution: 107240.00000000006
1149.9999999999418


## 
<div style="color:white;display:fill;border-radius:8px;
            background-color:#03112A;font-size:150%;
            letter-spacing:1.0px;background-image: url(https://i.imgur.com/GVd0La1.png)">
    <p style="padding: 8px;color:white;"><b><b><span style='color:#2ae4f5'>2.2 |</span></b> Financial Planning <a id = "4" ></b></p>
</div>

In [74]:
from pulp import *

In [86]:
# Step 1: Create the Optimization Model
model = LpProblem(name="FinancialPlanningAndControl", sense= LpMaximize)

In [87]:
# Step 2: Define Sets and Decision Variables
scenarios = [1, 2]
stage = [1, 2, 3]
invest = [1, 2]
pScenario = [0.5, 0.5, 0.5]
ReturnOfStock = [1.25, 1.06]
ReturnOfBond = [1.14, 1.12]
goal = 80000
InitialInvest = 55000
ShortagePenalty = 4
SurplusReward = 1

In [88]:
x1 = LpVariable.dicts("x1", [i for i in invest], lowBound=0, cat="continuous")
x2 = LpVariable.dicts("x2", [(i, s1) for i in invest for s1 in scenarios], lowBound=0, cat="continuous")
x3 = LpVariable.dicts("x3", [(i, s1, s2) for i in invest for s1 in scenarios for s2 in scenarios], lowBound=0, cat="continuous")
y = LpVariable.dicts("y", [(s1, s2, s3) for s1 in scenarios for s2 in scenarios for s3 in scenarios], lowBound=0, cat="continuous")
w = LpVariable.dicts("w", [(s1, s2, s3) for s1 in scenarios for s2 in scenarios for s3 in scenarios], lowBound=0, cat="continuous")

In [89]:
# Step 2: Define the Objective Function
model += lpSum(pScenario[s1 - 1] * pScenario[s2 - 1] * pScenario[s3 - 1] * SurplusReward * y[(s1, s2, s3)] for s1 in scenarios for s2 in scenarios for s3 in scenarios) - \
         lpSum(pScenario[s1 - 1] * pScenario[s2 - 1] * pScenario[s3 - 1] * ShortagePenalty * w[(s1, s2, s3)] for s1 in scenarios for s2 in scenarios for s3 in scenarios)

In [90]:
# Step 3: Add Constraints
model += lpSum(x1[i] for i in invest) == InitialInvest

for s in scenarios:
    model += ReturnOfStock[s - 1] * x1[1] + ReturnOfBond[s - 1] * x1[2] == x2[1, s] + x2[2, s]

for s1 in scenarios:
    for s2 in scenarios:
        model += ReturnOfStock[s2 - 1] * x2[1, s1] + ReturnOfBond[s2 - 1] * x2[2, s1] == x3[1, s1, s2] + x3[2, s1, s2]

for s1 in scenarios:
    for s2 in scenarios:
        for s3 in scenarios:
            model += ReturnOfStock[s3 - 1] * x3[1, s1, s2] + ReturnOfBond[s3 - 1] * x3[2, s1, s2] == goal + y[s1, s2, s3] - w[s1, s2, s3]

In [91]:
model.writeLP('FinancialPlanning.lp')

[w_(1,_1,_1),
 w_(1,_1,_2),
 w_(1,_2,_1),
 w_(1,_2,_2),
 w_(2,_1,_1),
 w_(2,_1,_2),
 w_(2,_2,_1),
 w_(2,_2,_2),
 x1_1,
 x1_2,
 x2_(1,_1),
 x2_(1,_2),
 x2_(2,_1),
 x2_(2,_2),
 x3_(1,_1,_1),
 x3_(1,_1,_2),
 x3_(1,_2,_1),
 x3_(1,_2,_2),
 x3_(2,_1,_1),
 x3_(2,_1,_2),
 x3_(2,_2,_1),
 x3_(2,_2,_2),
 y_(1,_1,_1),
 y_(1,_1,_2),
 y_(1,_2,_1),
 y_(1,_2,_2),
 y_(2,_1,_1),
 y_(2,_1,_2),
 y_(2,_2,_1),
 y_(2,_2,_2)]

In [82]:
model.solve(PULP_CBC_CMD(msg=False))

print("Status:", LpStatus[model.status])
print('Optimal Solution:', -value(model.objective))

Status: Optimal
Optimal Solution: 1514.0846500000007


In [83]:
# Shadow Prices
for var in model.variables():
    print(var, var.varValue)

w_(1,_1,_1) 0.0
w_(1,_1,_2) 0.0
w_(1,_2,_1) 0.0
w_(1,_2,_2) 0.0
w_(2,_1,_1) 0.0
w_(2,_1,_2) 0.0
w_(2,_2,_1) 0.0
w_(2,_2,_2) 12160.0
x1_1 41479.272
x1_2 13520.728
x2_(1,_1) 65094.582
x2_(1,_2) 36743.215
x2_(2,_1) 2168.138
x2_(2,_2) 22368.029
x3_(1,_1,_1) 83839.905
x3_(1,_1,_2) 0.0
x3_(1,_2,_1) 0.0
x3_(1,_2,_2) 64000.0
x3_(2,_1,_1) 0.0
x3_(2,_1,_2) 71428.571
x3_(2,_2,_1) 71428.571
x3_(2,_2,_2) 0.0
y_(1,_1,_1) 24799.881
y_(1,_1,_2) 8870.299
y_(1,_2,_1) 1428.5714
y_(1,_2,_2) 0.0
y_(2,_1,_1) 1428.5714
y_(2,_1,_2) 0.0
y_(2,_2,_1) -6.2359077e-12
y_(2,_2,_2) 0.0


In [85]:
# Shadow prices
for name, c in model.constraints.items():
    print(name, c, c.pi)

_C1 x1_1 + x1_2 = 55000 2.9444928
_C2 1.25*x1_1 + 1.14*x1_2 - x2_(1,_1) - x2_(2,_1) = -0.0 -0.922075
_C3 1.06*x1_1 + 1.12*x1_2 - x2_(1,_2) - x2_(2,_2) = -0.0 -1.6904708
_C4 1.25*x2_(1,_1) + 1.14*x2_(2,_1) - x3_(1,_1,_1) - x3_(2,_1,_1) = -0.0 -0.28875
_C5 1.06*x2_(1,_1) + 1.12*x2_(2,_1) - x3_(1,_1,_2) - x3_(2,_1,_2) = -0.0 -0.529375
_C6 1.25*x2_(1,_2) + 1.14*x2_(2,_2) - x3_(1,_2,_1) - x3_(2,_2,_1) = -0.0 -0.529375
_C7 1.06*x2_(1,_2) + 1.12*x2_(2,_2) - x3_(1,_2,_2) - x3_(2,_2,_2) = -0.0 -0.97052083
_C8 w_(1,_1,_1) + 1.25*x3_(1,_1,_1) + 1.14*x3_(2,_1,_1) - y_(1,_1,_1) = 80000.0 -0.125
_C9 w_(1,_1,_2) + 1.06*x3_(1,_1,_1) + 1.12*x3_(2,_1,_1) - y_(1,_1,_2) = 80000.0 -0.125
_C10 w_(1,_2,_1) + 1.25*x3_(1,_1,_2) + 1.14*x3_(2,_1,_2) - y_(1,_2,_1) = 80000.0 -0.125
_C11 w_(1,_2,_2) + 1.06*x3_(1,_1,_2) + 1.12*x3_(2,_1,_2) - y_(1,_2,_2) = 80000.0 -0.34542411
_C12 w_(2,_1,_1) + 1.25*x3_(1,_2,_1) + 1.14*x3_(2,_2,_1) - y_(2,_1,_1) = 80000.0 -0.125
_C13 w_(2,_1,_2) + 1.06*x3_(1,_2,_1) + 1.12*x3_(2,_2,_1